<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Machine Learning for Finance

## Basic Statistical Methods

Dr Yves J Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:ai@tpq.io">ai@tpq.io</a>

## Descriptive Statistics

In [ ]:
import numpy as np
import pandas as pd
from pylab import plt, mpl

In [ ]:
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
%config InlineBackend.figure_format = 'svg'

In [ ]:
url = 'https://certificate.tpq.io/mlfin.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0, parse_dates=True)
del raw['BTC=']
raw.dropna(inplace=True)

In [ ]:
rets = np.log(raw / raw.shift(1))

In [ ]:
rets.describe()

In [ ]:
rets.aggregate([np.min, np.mean, np.median, np.max])

In [ ]:
rets.iloc[:, :4].hist(bins=35);

In [ ]:
sym = 'AAPL.O'

In [ ]:
rets[sym].hist(bins=50);

In [ ]:
rets[sym].cumsum().apply(np.exp).plot();

In [ ]:
mean = rets[sym].mean()
mean

In [ ]:
std = rets[sym].std()
std

In [ ]:
tail = mean + 2 * std
tail

In [ ]:
tail_ = mean - 2 * std
tail_

In [ ]:
no = len(rets[sym][rets[sym].sort_values() > tail].values)
no

In [ ]:
plt.bar(np.arange(len(rets)),
        np.exp(rets.sort_values(sym)[sym].values.cumsum()));
plt.axvline(len(rets) - no, c='r', lw=1);

## Approximation

In [ ]:
def f(x):
    return np.sin(x) + 0.5 * x

In [ ]:
def create_plot(x, y, styles, labels, axlabels):
    plt.figure(figsize=(10, 6))
    for i in range(len(x)):
        plt.plot(x[i], y[i], styles[i], label=labels[i])
        plt.xlabel(axlabels[0])
        plt.ylabel(axlabels[1])
    plt.legend(loc=0)

In [ ]:
x = np.linspace(-2 * np.pi, 2 * np.pi, 50)  

In [ ]:
create_plot([x], [f(x)], ['b'], ['f(x)'], ['x', 'f(x)'])

### Regression

#### Monomials as Basis Functions

In [ ]:
res = np.polyfit(x, f(x), deg=1, full=True)  

In [ ]:
res  

In [ ]:
ry = np.polyval(res[0], x)  

In [ ]:
create_plot([x, x], [f(x), ry], ['b', 'r.'],
            ['f(x)', 'regression'], ['x', 'f(x)'])

In [ ]:
reg = np.polyfit(x, f(x), deg=5)
ry = np.polyval(reg, x)

In [ ]:
create_plot([x, x], [f(x), ry], ['b', 'r.'],
            ['f(x)', 'regression'], ['x', 'f(x)'])

In [ ]:
reg = np.polyfit(x, f(x), 7)
ry = np.polyval(reg, x)

In [ ]:
np.allclose(f(x), ry)  

In [ ]:
np.mean((f(x) - ry) ** 2)  

In [ ]:
create_plot([x, x], [f(x), ry], ['b', 'r.'],
            ['f(x)', 'regression'], ['x', 'f(x)'])

#### Individual Basis Functions

In [ ]:
matrix = np.zeros((3 + 1, len(x)))  
matrix[3, :] = x ** 3  
matrix[2, :] = x ** 2  
matrix[1, :] = x  
matrix[0, :] = 1  

In [ ]:
reg = np.linalg.lstsq(matrix.T, f(x), rcond=None)[0]  

In [ ]:
reg.round(4)  

In [ ]:
ry = np.dot(reg, matrix)  

In [ ]:
create_plot([x, x], [f(x), ry], ['b', 'r.'],
            ['f(x)', 'regression'], ['x', 'f(x)'])

In [ ]:
matrix[3, :] = np.sin(x)  

In [ ]:
reg = np.linalg.lstsq(matrix.T, f(x), rcond=None)[0]

In [ ]:
reg.round(4)  

In [ ]:
ry = np.dot(reg, matrix)

In [ ]:
np.allclose(f(x), ry)  

In [ ]:
np.mean((f(x) - ry) ** 2)  

In [ ]:
create_plot([x, x], [f(x), ry], ['b', 'r.'],
            ['f(x)', 'regression'], ['x', 'f(x)'])

#### Noisy Data

In [ ]:
xn = np.linspace(-2 * np.pi, 2 * np.pi, 50)  
xn = xn + 0.15 * np.random.standard_normal(len(xn))  
yn = f(xn) + 0.25 * np.random.standard_normal(len(xn))  

In [ ]:
reg = np.polyfit(xn, yn, 7)
ry = np.polyval(reg, xn)

In [ ]:
create_plot([x, x], [f(x), ry], ['b', 'r.'],
            ['f(x)', 'regression'], ['x', 'f(x)'])

#### Unsorted Data

In [ ]:
xu = np.random.rand(50) * 4 * np.pi - 2 * np.pi  
yu = f(xu)

In [ ]:
print(xu[:10].round(2))  
print(yu[:10].round(2))  

In [ ]:
reg = np.polyfit(xu, yu, 5)
ry = np.polyval(reg, xu)

In [ ]:
create_plot([xu, xu], [yu, ry], ['b.', 'ro'],
            ['f(x)', 'regression'], ['x', 'f(x)'])

#### Multiple Dimensions

In [ ]:
def fm(p):
    x, y = p
    return np.sin(x) + 0.25 * x + np.sqrt(y) + 0.05 * y ** 2

In [ ]:
x = np.linspace(0, 10, 20)
y = np.linspace(0, 10, 20)
X, Y = np.meshgrid(x, y)  

In [ ]:
Z = fm((X, Y))
x = X.flatten()  
y = Y.flatten()  

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = plt.subplot(projection='3d')
surf = ax.plot_surface(X, Y, Z, rstride=2, cstride=2,
                       cmap='coolwarm', linewidth=0.5,
                       antialiased=True)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f(x, y)')
fig.colorbar(surf, shrink=0.5, aspect=5);

In [ ]:
matrix = np.zeros((len(x), 6 + 1))
matrix[:, 6] = np.sqrt(y)  
matrix[:, 5] = np.sin(x)  
matrix[:, 4] = y ** 2
matrix[:, 3] = x ** 2
matrix[:, 2] = y
matrix[:, 1] = x
matrix[:, 0] = 1

In [ ]:
reg = np.linalg.lstsq(matrix, fm((x, y)), rcond=None)[0]

In [ ]:
RZ = np.dot(matrix, reg).reshape((20, 20))  

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10, 6))
surf1 = ax.plot_surface(X, Y, Z, rstride=2, cstride=2,
            cmap=mpl.cm.coolwarm, linewidth=0.5,
            antialiased=True)  
surf2 = ax.plot_wireframe(X, Y, RZ, rstride=2, cstride=2,
                          label='regression')  
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('f(x, y)')
ax.legend()
fig.colorbar(surf, shrink=0.5, aspect=5, ax=ax);

### Interpolation

In [ ]:
import scipy.interpolate as spi  

In [ ]:
x = np.linspace(-2 * np.pi, 2 * np.pi, 25)

In [ ]:
def f(x):
    return np.sin(x) + 0.5 * x

In [ ]:
ipo = spi.splrep(x, f(x), k=1)  

In [ ]:
iy = spi.splev(x, ipo)  

In [ ]:
np.allclose(f(x), iy)  

In [ ]:
create_plot([x, x], [f(x), iy], ['b', 'ro'],
            ['f(x)', 'interpolation'], ['x', 'f(x)'])

In [ ]:
xd = np.linspace(1.0, 3.0, 50)  
iyd = spi.splev(xd, ipo)

In [ ]:
create_plot([xd, xd], [f(xd), iyd], ['b', 'ro'],
            ['f(x)', 'interpolation'], ['x', 'f(x)'])

In [ ]:
ipo = spi.splrep(x, f(x), k=3)  
iyd = spi.splev(xd, ipo)  

In [ ]:
np.allclose(f(xd), iyd)  

In [ ]:
np.mean((f(xd) - iyd) ** 2)  

In [ ]:
create_plot([xd, xd], [f(xd), iyd], ['b', 'ro'],
            ['f(x)', 'interpolation'], ['x', 'f(x)'])

## Pattern Frequencies

In [ ]:
f = 10
n = 25000

In [ ]:
np.random.seed(100)

In [ ]:
x = np.random.randint(0, 2, (n, f))
x[:4]

In [ ]:
y = np.random.randint(0, 2, n)
y[:4]

In [ ]:
2 ** f

In [ ]:
fcols = [f'f{_}' for _ in range(f)]
fcols

In [ ]:
data = pd.DataFrame(x, columns=fcols)
data['l'] = y

In [ ]:
data.info()

In [ ]:
grouped = data.groupby(list(data.columns))

In [ ]:
freq = grouped['l'].size().unstack(fill_value=0)

In [ ]:
freq['sum'] = freq[0] + freq[1]

In [ ]:
freq.head(10)

In [ ]:
freq['sum'].describe().astype(int)

<img src='http://hilpisch.com/tpq_logo.png' width="35%" align="right">

<br><br><a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:ai@tpq.io">ai@tpq.io</a>